In [1]:
import xlsxwriter
import pandas as pd
import openpyxl
import re

In [2]:
raw_data_file = "data/raw/Google 3.0.xlsx"

In [3]:
raw_df = pd.read_excel(open(raw_data_file, 'rb'), sheet_name="6 month")
taf_df = raw_df

In [4]:
def split_tags(tag_str):
    tag_str = re.sub(r"([a-z])([A-Z])", r"\1@\2", tag_str)
    tag_str = tag_str.replace(")", ")@")
    return tag_str.split("@")


In [5]:
taf_df["processed_tags"] = taf_df["Tags"].apply(lambda row: split_tags(row))
all_tags = set(t for tags in taf_df["processed_tags"] for t in tags)

In [6]:
# all_tags

In [7]:
wb = openpyxl.load_workbook(raw_data_file)
ws = wb.get_sheet_by_name('6 month')

/var/folders/w_/fx6_90ns0933s65jzp2cx5zc0000gn/T/ipykernel_56702/77425423.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws = wb.get_sheet_by_name('6 month')


In [8]:
ws.cell(row=2, column=3).hyperlink.display

In [9]:
def separate_tags_in_ws(wsheet, row_tags):
    for i, (key, *values) in enumerate(wsheet.iter_rows()):
        if i > 0 and i - 1 < len(row_tags):
            values[1].value = '; '.join(row_tags.iloc[i - 1])
    return wsheet

In [10]:
new_ws = separate_tags_in_ws(ws, taf_df["processed_tags"]) #.cell(row=2, column=2).hyperlink.target

In [11]:
# taf_df

In [12]:
tab_tags = {
    'Array': 'Array',
    'Backtracking': 'Backtracking_Recursion',
    'Binary Indexed Tree': 'Binary Indexed Tree',
    'Binary Search': 'Binary Search',
    'Binary Search Tree': 'Tree',
    'Binary Tree': 'Tree',
    'Bit Manipulation': 'Other',
    'Bitmask': 'Other',
    'Brainteaser': 'Other',
    'Breadth-First Search': 'BFS',
    'Bucket Sort': 'Other',
    'Combinatorics': 'Other',
    'Counting': 'Other',
    'Data Stream': 'Other',
    'Database': 'Other',
    'Depth-First Search': 'DFS',
    'Design': 'Design',
    'Divide and Conquer': 'Divide and Conquer',
    'Doubly-Linked List': 'Linked List',
    'Dynamic Programming': 'DP',
    'Enumeration': 'Other',
    'Eulerian Circuit': 'Other',
    'Game Theory': 'Other',
    'Geometry': 'Other',
    'Graph': 'Graph',
    'Greedy': 'Other',
    'Hash Function': 'Hash Table',
    'Hash Table': 'Hash Table',
    'Heap (Priority Queue)': 'Heap',
    'Interactive': 'Other',
    'Iterator': 'Other',
    'Line Sweep': 'Segment Tree',
    'Linked List': 'Linked List',
    'Math': 'Other',
    'Matrix': 'Other',
    'Memoization': 'DP',
    'Merge Sort': 'Sorting',
    'Monotonic Queue': 'Stack_Queue',
    'Monotonic Stack': 'Stack_Queue',
    'Ordered Set': 'Hash Table',
    'Prefix Sum': 'Hash Table',
    'Probability and Statistics': 'Other',
    'Queue': 'Stack_Queue',
    'Quickselect': 'Sorting',
    'Randomized': 'Other',
    'Recursion': 'Backtracking_Recursion',
    'Rolling Hash': 'Hash Table',
    'Segment Tree': 'Segment Tree',
    'Shortest Path': 'Other',
    'Simulation': 'Other',
    'Sliding Window': 'Sliding Window',
    'Sorting': 'Sorting',
    'Stack': 'Stack_Queue',
    'String': 'String',
    'String Matching': 'String',
    'Suffix Array': 'Other',
    'Topological Sort': 'Topological Sort',
    'Tree': 'Tree',
    'Trie': 'Trie',
    'Two Pointers': 'Two Pointers',
    'Union Find': 'Union Find',
}

In [13]:
ordering = [
    'DP',
    'Binary Search',
    'Heap',
    'DFS',
    'BFS',
    'Hash Table',
    'Backtracking_Recursion',
    'Two Pointers',
    'Topological Sort',
    'Trie',
    'Stack_Queue',
    'Linked List',
    'Union Find',
    'Sliding Window',
    'Divide and Conquer',
    'Graph',
    'Tree',
    'Binary Indexed Tree',
    'Design',
    'Array',
    'Segment Tree',
    'Sorting',
    'String',
    'Other',
]

ordering_map = {tag: i for i, tag in enumerate(ordering)}

In [14]:
def create_new_sheet_based_on_tags(wbook):
    all_tabs_names = set(tab_tags.values())
    for tab_name in all_tabs_names:
        wbook.create_sheet(tab_name)
    # return wbook

In [15]:
create_new_sheet_based_on_tags(wb)

In [16]:
len(wb.sheetnames), len(set(tab_tags.values()))

(25, 24)

In [17]:
from copy import copy 
def distribute_to_corresponding_tabs(ws, wb, row_tags):
    for i, row in enumerate(ws):
        if i > 0 and i - 1 < len(row_tags):
            sorted_tags = sorted(map(lambda x: tab_tags.get(x, ''), row_tags.iloc[i - 1]), key=lambda x: ordering_map.get(x, 24))
            top_tag = sorted_tags[0]
            sheet = wb[top_tag]
            
            selected_row = [cell.value for cell in row]
            sheet.append(selected_row)
            for j, cell in enumerate(row):
                col = j+1
                new_cell = sheet.cell(row=sheet.max_row, column=col)
                if cell.has_style:
                    new_cell.hyperlink = copy(cell.hyperlink)
                    new_cell.font = copy(cell.font)
                    new_cell.border = copy(cell.border)
                    new_cell.fill = copy(cell.fill)
                    new_cell.number_format = copy(cell.number_format)
                    new_cell.protection = copy(cell.protection)
                    new_cell.alignment = copy(cell.alignment)
                    
    for sheet_name in list(wb.sheetnames)[1:]:
        for col, value in wb['6 month'].column_dimensions.items():
            wb[sheet_name].column_dimensions[col] = value

distribute_to_corresponding_tabs(ws, wb, taf_df["processed_tags"])

In [18]:
# wb.sheetnames

In [20]:
wb.save("data/processed/Google 3.0_12222021.xlsx")